# Summer of Code - Artificial Intelligence

## Week 10: Deep Learning

### Day 05: Machine Translation

In this notebook, we will explore **Machine Translation** using **Bidirectional RNNs** in PyTorch.

# Neural Machine Translation
Neural Machine Translation (NMT) is another NLP task where we translate text from one language to another using neural networks.